In [20]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
import numpy
import optuna as optuna

C:\Users\Dell\anaconda3\envs\car_damage_detection\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Load Data

In [3]:
image_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406],std =[0.229,0.224,0.225], )
])

In [4]:
dataset_path = "./dataset"
dataset = datasets.ImageFolder(root = dataset_path, transform=image_transform)

In [5]:
len(dataset)

2301

In [6]:
dataset.classes

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [7]:
num_classes = len(dataset.classes)
num_classes

6

In [8]:
train_size = int(0.75 * len(dataset))
val_size = len(dataset) - train_size

train_size, val_size

(1725, 576)

In [9]:
from torch.utils.data import random_split

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [10]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=True)

In [11]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [12]:
images[0].shape

torch.Size([3, 224, 224])

In [13]:
labels[0]

tensor(3)

In [14]:
images[0].permute(1,2,0).shape

torch.Size([224, 224, 3])

In [15]:
# plt.imshow(images[14].permute(1,2,0))
# plt.show()

In [16]:
labels[14]

tensor(5)

### Model training and hyperparameter tuning

In [17]:
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate = 0.5):
        super().__init__()
        
        self.model = models.resnet50(weights="DEFAULT")

        # freeze all layers except the final fully connected layers
        for param in self.model.parameters():
            param.requires_grad = False

        # unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        # replace the fully connected layer
        self.model.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features , num_classes)
            )

    def forward(self,x):
        x = self.model(x)
        return x 

In [24]:
def objective(trial):

    # suggest values for hyperparameter tuning

    lr = trial.suggest_float("lr", 1e-5, 1e-2, log = True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2,0.7)

    # load the model
    model = CarClassifierResNet(num_classes = num_classes, dropout_rate=dropout_rate).to(device)

    # define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr = lr)

    # training loop
    epochs = 3
    start = time.time()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)

            loss = criterion(outputs,labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation loop
        model.eval()
        correct = 0
        total = 0 
        with torch.no_grad():
            for images,labels in val_loader:
                mages, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data,1)
                total += labels.size(0)
                correct += (predicted == labels). sum().item()

        accuracy = 100 * correct / total

        # report intermediate result to optuna
        trial.report(accuracy, epoch)

        # Handle pruning

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time : {end - start} seconds")

    return accuracy
    

In [25]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 20)

[I 2025-08-03 10:39:22,648] A new study created in memory with name: no-name-c9775336-b0ec-4ef1-86b5-b2935f8f61df
[I 2025-08-03 11:14:59,491] Trial 0 finished with value: 46.18055555555556 and parameters: {'lr': 1.8535370216250005e-05, 'dropout_rate': 0.6175570776995238}. Best is trial 0 with value: 46.18055555555556.


Execution time : 2135.529582977295 seconds


[I 2025-08-03 11:42:44,744] Trial 1 finished with value: 36.111111111111114 and parameters: {'lr': 1.5004231446747878e-05, 'dropout_rate': 0.5072647229197326}. Best is trial 0 with value: 46.18055555555556.


Execution time : 1663.5626606941223 seconds


[I 2025-08-03 12:08:17,874] Trial 2 finished with value: 80.55555555555556 and parameters: {'lr': 0.000570610825826787, 'dropout_rate': 0.5560482291947226}. Best is trial 2 with value: 80.55555555555556.


...

[I 2025-08-03 14:54:09,268] Trial 12 pruned. 
[I 2025-08-03 15:14:08,477] Trial 13 finished with value: 81.25 and parameters: {'lr': 0.009270814900525432, 'dropout_rate': 0.56256113867564}. Best is trial 11 with value: 82.29166666666667.


Execution time : 1198.6858332157135 seconds


[I 2025-08-03 15:27:33,654] Trial 14 pruned. 
[I 2025-08-03 15:47:42,481] Trial 15 finished with value: 80.72916666666667 and parameters: {'lr': 0.0030170154912768613, 'dropout_rate': 0.4885893371886448}. Best is trial 11 with value: 82.29166666666667.


Execution time : 1208.0942821502686 seconds


[I 2025-08-03 16:01:02,775] Trial 16 pruned. 
[I 2025-08-03 16:14:19,166] Trial 17 pruned. 
[I 2025-08-03 16:27:36,426] Trial 18 pruned. 
[I 2025-08-03 16:34:18,399] Trial 19 pruned. 


In [26]:
  # Assuming 'study' is your Optuna study object after optimization
best_trial = study.best_trial
print(f"Best trial value: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")

Best trial value: 82.29166666666667
Best hyperparameters: {'lr': 0.007609280382829907, 'dropout_rate': 0.5302808827244218}
